In [ ]:
import pybamm
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
soc_grid = np.linspace(0.0, 1.0, 100)
ocv_values = []

In [ ]:
def voltage_from_soc(soc):
    soc = np.clip(soc, 0.0, 1.0)
    return ocv_poly(soc)

In [ ]:
for soc in soc_grid:
    model = pybamm.lithium_ion.SPM()
    param = pybamm.ParameterValues("Chen2020")
    experiment = pybamm.Experiment(["Rest for 1 hour"]) # no current at rest

    sim = pybamm.Simulation(
        model,
        parameter_values=param,
        experiment=experiment
    )

    sol = sim.solve(initial_soc=soc)

    ocv = sol["Terminal voltage [V]"].entries[-1]
    ocv_values.append(ocv)

ocv_values = np.array(ocv_values)

coeffs = np.polyfit(soc_grid, ocv_values, deg=5)
ocv_poly = np.poly1d(coeffs)

In [ ]:
plt.figure(figsize=(7, 5))
plt.plot(soc_grid, ocv_values, label="PyBaMM OCV", lw=2)
plt.plot(soc_grid, ocv_poly(soc_grid), "--", label="5th-degree Polynomial Fit", lw=2)
plt.xlabel("State of Charge (SOC)")
plt.ylabel("Voltage (V)")
plt.title("OCV vs SOC (PyBaMM – correct method)")
plt.grid(True)
plt.legend()
plt.show()